In [1]:
import subprocess
import os
from tqdm import tqdm

dir_path = '/workspace/AVE_Dataset/AVE/'
pro_path = '/workspace/AVE_Dataset/AVE_processed/'
files = os.listdir(dir_path)
print(len(files))

os.makedirs(pro_path, exist_ok=True)

def process_video(input_path: str, output_path: str):
    cmd = [
        "ffmpeg",
        "-y",  # 기존 파일 덮어쓰기
        "-i", input_path,  # 입력 파일
        "-t", "10",  # 앞 5초만 추출
        "-vf", "crop='min(in_w, in_h)':'min(in_w, in_h)',scale=128:128,fps=16",  # 중앙 crop 후 resize + fps
        "-an",  # 오디오 제거 (필요 시 제거)
        output_path
    ]
    
    subprocess.run(cmd, check=True)

def process_video(input_path: str, output_path: str):
    cmd = [
        "ffmpeg",
        "-y",
        "-i", input_path,
        "-t", "10",
        "-vf", (
            "crop='min(in_w\\,in_h)':'min(in_w\\,in_h)':"
            "(in_w - min(in_w\\,in_h))/2:"
            "(in_h - min(in_w\\,in_h))/2,"
            "scale=128:128,"
            "fps=24"
        ),
        output_path
    ]

    import subprocess
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

4097


In [2]:
import os
from multiprocessing import Pool
from tqdm import tqdm

def process_video_wrapper(args):
    input_path, output_path = args
    try:
        process_video(input_path, output_path)
    except Exception as e:
        print("ERror : ", e)

def process_all_videos(files, dir_path, pro_path):
    args_list = [
        (os.path.join(dir_path, fp), os.path.join(pro_path, fp[:-4] + "_126res_24fps.mp4"))
        for fp in files
    ]

    with Pool(processes=6) as pool:
        list(tqdm(pool.imap_unordered(process_video_wrapper, args_list), total=len(args_list)))

print('시작')
process_all_videos(files, dir_path, pro_path)

시작


100%|██████████| 4097/4097 [27:37<00:00,  2.47it/s]


In [6]:
import ffmpeg
import os
from multiprocessing import Pool
from tqdm import tqdm

dir_path = '/workspace/AVE_Dataset/AVE_processed/'
files = os.listdir(dir_path)
audio_path = '/workspace/AVE_Dataset/AVE_audio/'
os.makedirs(audio_path, exist_ok=True)

def extract_audio(input_video_path, output_audio_path):
    (
        ffmpeg
        .input(input_video_path)
        .output(output_audio_path, format='wav', acodec='pcm_s16le', ac=1, ar='44100')
        .run(quiet=True)
    )

def process_audio_wrapper(args):
    input_path, output_path = args
    try:
        extract_audio(input_path, output_path)
    except Exception as e:
        print("Error : ", e)

def process_all_audios(files, dir_path, pro_path):
    args_list = [
        (os.path.join(dir_path, fp), os.path.join(pro_path, fp[:-4] + ".wav"))
        for fp in files
    ]

    with Pool(processes=4) as pool:
        list(tqdm(pool.imap_unordered(process_audio_wrapper, args_list), total=len(args_list)))

print('시작')
process_all_audios(files, dir_path, audio_path)
# extract_audio(dir_path+files[0], 'output_test.wav')

시작


100%|██████████| 4097/4097 [01:34<00:00, 43.38it/s]


In [5]:
files[0]

'_tGdoqj0Qh8.mp4'

In [ ]:
from datasets import load_dataset

ds = load_dataset("Loie/VGGSound")